#### IMPORT NECESSARY LIBRARIES

In [109]:
from sklearn.svm import SVR #Regression Model
import pandas as pd # For data wrangling
from sklearn.model_selection import train_test_split # For data splitting
from sklearn.preprocessing import StandardScaler # For normal distribution of the data 
from sklearn.metrics import mean_squared_error,r2_score # For model evaluation
from mealpy.evolutionary_based.GA import BaseGA # metaheuristic algorithm for optimization and hyperparameter tuning
import numpy as np # for mathematical manipulation

In [110]:
data=pd.read_excel(r"C:\Users\USER\Desktop\Biodiesel Yield\data 1.xlsx")
data.head()

,X1,X2,X3,X4,Y
0,80,5,105,10.5,51.68
1,60,3,105,10.5,59.89
2,60,3,105,10.5,61.09
3,40,3,60,10.5,49.56
4,40,3,150,10.5,58.02


#### SEPARATING DATA INTO FEATURES AND TARGET

In [111]:
X=data.drop(columns="Y")
y=data["Y"]

In [112]:
X1=X.values
y1=y.values.reshape(-1,1)
X_train1, X_test1, y_train1, y_test1= train_test_split(X1, y1, test_size=0.2, random_state=8)

sc=StandardScaler() # For normal distribution of data
X_train1=sc.fit_transform(X_train1)
X_test1=sc.transform(X_test1)
x_df=sc.transform(X1)

sc_y=StandardScaler()
y_train1=sc_y.fit_transform(y_train1)
y_test1=sc_y.transform(y_test1)
y_df=sc_y.transform(y1)

print(X_train1.shape)
print(y_train1.shape)

(23, 4)
(23, 1)


In [171]:
# Define the SVR fitness function
def svr_fitness(solution):
    kernel_encoded = int(solution[0])
    c = solution[1]
    gamma = solution[2]
    epsilon = solution[3]
    
    kernel_decoded = ['linear', 'poly', 'rbf', 'precomputed'][kernel_encoded]
    
    svr = SVR(C=c, kernel=kernel_decoded, gamma=gamma, epsilon=epsilon)
    svr.fit(X_train1, y_train1.ravel())
    y_predict = svr.predict(X_test1)
    fitness = -r2_score(y_test1, y_predict)  # Minimize negative R^2 score
    return fitness

In [172]:
# Define the hyperparameter bounds
param_bounds = {
    "kernel": [0, 1, 2, 3],  # 0: linear, 1: poly, 2: rbf 3: precomputed
    "C": list(range(100, 1001, 100)),
    "gamma": list(np.logspace(-10, 0, 10)),
    "epsilon": list(np.logspace(-3, 0, 10))
}

In [182]:
# Define the optimization problem dictionary
svr_problem = {
    "fit_func": svr_fitness,
    "lb": [min(param_bounds["kernel"]), min(param_bounds["C"]), min(param_bounds["gamma"]), min(param_bounds["epsilon"])],
    "ub": [max(param_bounds["kernel"]), max(param_bounds["C"]), max(param_bounds["gamma"]), max(param_bounds["epsilon"])],
    "minmax": "min"  # Minimize fitness (negative R^2)
}

# lb : lower boundary of hyperparameter values
# ub : upper boundary of hyperparameter values
# fit_func : Fitness Function, SVR(), in this case.
# minmax : minimization and maximization

In [183]:
# Create the genetic algorithm model using mealpy

epoch = 50 # No of epochs/iterations
pop_size = 100 # Population size
pc = 0.95 # probabilty of crossover
pm = 0.05 # probability of mutation
ga_model = BaseGA(epoch, pop_size, pc, pm)

In [175]:
# Solve the optimization problem using the genetic algorithm
best_params, best_fitness = ga_model.solve(svr_problem)

2023/08/08 09:52:33 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.
2023/08/08 09:52:38 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 1, Current best: -0.9307344690034542, Global best: -0.9307344690034542, Runtime: 2.57250 seconds
2023/08/08 09:52:39 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 2, Current best: -0.930734469003473, Global best: -0.930734469003473, Runtime: 0.93901 seconds
2023/08/08 09:52:40 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 3, Current best: -0.9353459375974014, Global best: -0.9353459375974014, Runtime: 1.27198 seconds
2023/08/08 09:52:40 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 4, Current best: -0.9358606377323686, Global best: -0.9358606377323686, Runtime: 0.47774 seconds
2023/08/08 09:52:41 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 5, Current best: -0.9359828319024812, Global best: -0.93598283190

2023/08/08 09:52:53 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 45, Current best: -0.978509667165826, Global best: -0.978509667165826, Runtime: 0.61061 seconds
2023/08/08 09:52:54 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 46, Current best: -0.978509667165826, Global best: -0.978509667165826, Runtime: 0.44779 seconds
2023/08/08 09:52:54 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 47, Current best: -0.978509667165826, Global best: -0.978509667165826, Runtime: 0.18953 seconds
2023/08/08 09:52:54 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 48, Current best: -0.978509667165826, Global best: -0.978509667165826, Runtime: 0.42666 seconds
2023/08/08 09:52:55 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 49, Current best: -0.978509667165826, Global best: -0.978509667165826, Runtime: 0.33328 seconds
2023/08/08 09:52:55 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch

In [176]:
best_kernel_encoded = int(best_params[0])
best_c = best_params[1]
best_gamma = best_params[2]
best_epsilon = best_params[3]
best_kernel_decoded = ['linear', 'poly', 'rbf', 'precomputed'][best_kernel_encoded]

In [177]:
print("Best Hyperparameters:")
print("Kernel:", best_kernel_decoded)
print("C:", best_c)
print("Gamma:", best_gamma)
print("Epsilon:", best_epsilon)
print("Best Fitness (R^2):", -best_fitness)  # Convert back to positive (R^2)

Best Hyperparameters:
Kernel: rbf
C: 813.6090169835028
Gamma: 0.021132010324648277
Epsilon: 0.03977309223580246
Best Fitness (R^2): 0.978509667165826


#### Now, let's Create our SVR() using the MEALPY GA optimized hyperparameters

In [184]:
svr_optimized = SVR(kernel=best_kernel_decoded,C=best_c, gamma =best_gamma,epsilon = best_epsilon)

In [185]:
svr_optimized.fit(X_train1, y_train1.ravel())

SVR(C=813.6090169835028, epsilon=0.03977309223580246,
    gamma=0.021132010324648277)

In [186]:
pred_test = svr.predict(X_test1)

In [208]:
from sklearn.metrics import r2_score
print("R^2 score of Test set prediction using GA:", r2_score(y_test1,pred_test))

R^2 score of Test set prediction using GA: 0.978509667165826
R^2 score of Train set prediction using GA: 0.9789348583806831


In [205]:
pred_train = svr.predict(X_train1)
print("R^2 score of Train set prediction using GA:", r2_score(y_train1,pred_train))

R^2 score of Train set prediction using GA: 0.9789348583806831


#### Now that we've seen how useful and powerful the genetic algorithm of the MEALPY library is, lets also try out one more optimization algorithm, Particle Swarm Optimization (PSO)

In [189]:
from mealpy.swarm_based.PSO import CL_PSO #importing the PSO Library

In [190]:
epoch = 50
pop_size = 100
c_local = 1.2
w_min = 0.4
w_max = 0.9
max_flag = 7

In [194]:
pso_model = CL_PSO(epoch, pop_size, c_local, w_min, w_max, max_flag)

In [195]:
# Solve the optimization problem using the PSO algorithm
best_params_pso, best_fitness_pso = pso_model.solve(svr_problem)

2023/08/08 10:15:24 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: Solving single objective optimization problem.
2023/08/08 10:15:27 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 1, Current best: -0.8551661748697635, Global best: -0.8551661748697635, Runtime: 1.50302 seconds
2023/08/08 10:15:29 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 2, Current best: -0.8551661748697635, Global best: -0.8551661748697635, Runtime: 1.38361 seconds
2023/08/08 10:15:30 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 3, Current best: -0.8551661748697635, Global best: -0.8551661748697635, Runtime: 1.45136 seconds
2023/08/08 10:15:32 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 4, Current best: -0.8551661748697635, Global best: -0.8551661748697635, Runtime: 1.51826 seconds
2023/08/08 10:15:34 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 5, Current best: -0.8551661748697635, Global best: -0.8551661748697635, Runtime: 2.03275 seconds
20

2023/08/08 10:16:54 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 47, Current best: -0.9719079046215082, Global best: -0.9719079046215082, Runtime: 4.36036 seconds
2023/08/08 10:16:57 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 48, Current best: -0.9719079046215082, Global best: -0.9719079046215082, Runtime: 3.30097 seconds
2023/08/08 10:16:59 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 49, Current best: -0.9719079046215082, Global best: -0.9719079046215082, Runtime: 2.38395 seconds
2023/08/08 10:17:03 AM, INFO, mealpy.swarm_based.PSO.CL_PSO: >Problem: P, Epoch: 50, Current best: -0.9719079046215082, Global best: -0.9719079046215082, Runtime: 3.13199 seconds


In [196]:
pso_best_kernel_encoded = int(best_params_pso[0])
pso_best_c = best_params_pso[1]
pso_best_gamma = best_params_pso[2]
pso_best_epsilon = best_params_pso[3]
pso_best_kernel_decoded = ['linear', 'poly', 'rbf', 'precomputed'][pso_best_kernel_encoded]

In [197]:
print("PSO Best Hyperparameters:")
print("Kernel:", pso_best_kernel_decoded)
print("C:", pso_best_c)
print("Gamma:", pso_best_gamma)
print("Epsilon:", pso_best_epsilon)
print("Best Fitness (R^2):", -best_fitness_pso)  # Convert back to positive (R^2)

PSO Best Hyperparameters:
Kernel: rbf
C: 675.3082014594984
Gamma: 0.030145945579186106
Epsilon: 0.025345863350150526
Best Fitness (R^2): 0.9719079046215082


#### Now, let's Create our SVR() using the MEALPY PSO optimized hyperparameters

In [199]:
svr_optimized_pso = SVR(kernel=pso_best_kernel_decoded,C=pso_best_c, gamma = pso_best_gamma,epsilon = pso_best_epsilon)

In [200]:
svr_optimized_pso.fit(X_train1, y_train1.ravel())

SVR(C=675.3082014594984, epsilon=0.025345863350150526,
    gamma=0.030145945579186106)

In [201]:
pred_test_pso = svr_optimized_pso.predict(X_test1)

In [202]:
from sklearn.metrics import r2_score
print("R^2 score of test set prediction using PSO :", r2_score(y_test1,pred_test_pso))

R^2 score of test set prediction using PSO : 0.9719079046215082


In [210]:
pred_train_pso = svr_optimized_pso.predict(X_train1)
print("R^2 score of Train set prediction using PSO:", r2_score(y_train1,pred_train_pso))

R^2 score of Train set prediction using PSO: 0.978840873734026


### Yeah, Good Performance, a proof of effective hyperparameter tuning by Metaheuristic algorithms.